# PRV Assignment 1

- Pablo Garcia Fernández

Initial configuration

In [70]:
# Initial configuration
import numpy as np
from math import cos,sin

#######################################
## PREDIFINE CONSTATNS OF THE EXERCISE

ROLL=2.5; PITCH=5; YAW=60 
TRANSLATION = np.array([0, 0, 0.25])

#F = 2406.1
F = 3.62 # focal length (mm)
CX = 25.498; CY=-33.0153
K1 = 0.00247448; K2 = -0.0240244; K3 = 0.0604812; K4 = -0.0275491; 
P1 = 0.000150734; P2 = -0.000271032
B1 = -5.67637; B2 = 0.339356
PIXEL_SIZE = 0.00156  # (mm)
RESOLUTION_W = 4000   # (px)
RESOLUTION_H = 3000   # (PX)

W = RESOLUTION_W * PIXEL_SIZE  # (mm)
H = RESOLUTION_H * PIXEL_SIZE  # (mm)
######################################

Auxiliar functions

In [71]:
def enu_to_ned_ned_to_enu():
    transformation_matrix = np.array([
        [0., 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, -1, 0],
		[0, 0, 0, 1]
    ])
    return transformation_matrix


def _3D_rotation_matrix(roll, pitch, yaw, order='ypr', inverso=False):
	# Return the rotation matrix in 3D coordinate systems
	yaw = np.array([
		[cos(yaw), -sin(yaw), 0, 0],
		[sin(yaw), cos(yaw), 0, 0],
		[0, 0, 1, 0],
		[0, 0, 0, 1]
	])
	pitch = np.array([
		[cos(pitch), 0, sin(pitch), 0],
		[0, 1, 0, 0],
		[-sin(pitch), 0, cos(pitch), 0],
		[0, 0, 0, 1]
	])
	roll = np.array([
		[1, 0, 0, 0],
		[0, cos(roll), -sin(roll), 0],
		[0, sin(roll), cos(roll), 0],
		[0, 0, 0, 1]
	]) 

	rotation_matrix = None
	
	if order=='ypr':
		if inverso:
			rotation_matrix = yaw.T @ pitch.T @ roll.T
		else:
			rotation_matrix = roll @ pitch @ yaw
	elif order == 'rpy':
		if inverso:
			rotation_matrix = roll.T @ pitch.T @ yaw.T
		else:
			rotation_matrix = yaw @ pitch @ roll
	else:
		print("Order not supported")

	return rotation_matrix

### World CS -> INS CS
### INS CS -> INS CS

In [72]:
def ins_to_wcs():
    # Note -> WCS is ENU
        
    # 1) yaw, pitch, roll rotation
    t1 = _3D_rotation_matrix(ROLL, PITCH, YAW)
    
    # 2) NED to ENU
    t2 = enu_to_ned_ned_to_enu()
    
    return t2 @ t1

def wcs_to_ins():
    # 1) ENU to NED
    t1 = enu_to_ned_ned_to_enu()

    # 2) yaw, pitch, roll rotation
    t2 = _3D_rotation_matrix(ROLL, PITCH, YAW, inverso=True)

    return t2 @ t1

### INS CS -> Gimbal CS
### Gimbal CS -> INS CS 

In [73]:
def gimbal_to_ins(traslation_vector):
    # 1) Translation + 2) ENU to NED
    transformation_matrix = enu_to_ned_ned_to_enu()
    transformation_matrix[:-1,-1] = traslation_vector

    return transformation_matrix

def ins_to_gimbal(traslation_vector):
    # 1) NED to ENU + 2) - translation
    transformation_matrix = enu_to_ned_ned_to_enu()
    transformation_matrix[:-1,-1] = traslation_vector

    return transformation_matrix


### Gimbal -> Camera
### Camera -> Gimbal

In [74]:
def camera_to_gimbal():
	# Frontal to nadiral
	# Rotation -90º axis X
	t = np.array([
		[1, 0, 0, 0],
		[0, 0, 1, 0],
		[0, -1, 0, 0],
		[0, 0, 0, 1]
	]) 
	return t

def gimbal_to_camera():
	# Nadiral to frontal
	# Rotation 90º axis X
    t = np.array([
		[1, 0, 0, 0],
		[0, 0, -1, 0],
		[0, 1, 0, 0],
		[0, 0, 0, 1]
	]) 
    return t

### Composition of all transformations

In [75]:
# 1st path: camera to wcs
t1 = camera_to_gimbal()
print("camera to gimbal:\n", t1)
t2 = gimbal_to_ins(TRANSLATION)
print("gimbal to ins:\n", t2)
t3 = ins_to_wcs()
print("ins to wcs:\n",t3)
# TOTAL
t = t3 @ t2 @ t1
print("full matrix:", t)
### Ejemplo de proyeccion
punto = np.array([3, 4, 2, 1])
proyeccion = t @ punto
print("Proyeccion del punto", punto, "a ", proyeccion)
print("\n\n")

# Camino 2 de wcs a camera
t1 = wcs_to_ins()
print("wcs to ins:\n", t1)
t2 = ins_to_gimbal(TRANSLATION)
print("ins to gimbal:\n", t2)
t3 = gimbal_to_camera()
print("gimbal to camera:\n",t3)
# TOTAL
t = t3 @ t2 @ t1
print("full matrix:", t)
### Ejemplo de proyeccion
punto = np.array([2.624333, -4.35636552, 2.27979292, 1. ])
proyeccion = t @ punto
print("Proyeccion del punto", punto, "a ", proyeccion)

camera to gimbal:
 [[ 1  0  0  0]
 [ 0  0  1  0]
 [ 0 -1  0  0]
 [ 0  0  0  1]]
gimbal to ins:
 [[ 0.    1.    0.    0.  ]
 [ 1.    0.    0.    0.  ]
 [ 0.    0.   -1.    0.25]
 [ 0.    0.    0.    1.  ]]
ins to wcs:
 [[ 0.79077686  0.58809197 -0.16976392  0.        ]
 [-0.27016355  0.08646325 -0.95892427  0.        ]
 [-0.54925733  0.80415915  0.22725415  0.        ]
 [ 0.          0.          0.          1.        ]]
full matrix: [[ 0.58809197 -0.16976392  0.79077686 -0.04244098]
 [ 0.08646325 -0.95892427 -0.27016355 -0.23973107]
 [ 0.80415915  0.22725415 -0.54925733  0.05681354]
 [ 0.          0.          0.          1.        ]]
Proyeccion del punto [3 4 2 1] a  [ 2.624333   -4.35636552  2.27979292  1.        ]



wcs to ins:
 [[ 0.79077686 -0.27016355 -0.54925733  0.        ]
 [ 0.58809197  0.08646325  0.80415915  0.        ]
 [-0.16976392 -0.95892427  0.22725415  0.        ]
 [ 0.          0.          0.          1.        ]]
ins to gimbal:
 [[ 0.    1.    0.    0.  ]
 [ 1.    0.

## CAMERA MODEL

In [76]:
def camera_model(punto):
    X, Y, Z = punto[0], punto[1], punto[2]
    
    x = X/Z
    y = Y/Z
    r = np.sqrt(x**2 + y**2)
    
    x_prima = x * (1 + K1*r**2 + K2*r**4 + K3*r**6 + K4*r**8 + 
        (P1*(r**2+2*x**2) + 2*P2*x*y))
    y_prima = y * (1 + K1*r**2 + K2*r**4 + K3*r**6 + K4*r**8 + 
        (P2*(r**2+2*y**2) + 2*P1*x*y))
    
    u = W * 0.5 + CX + x_prima*F + x_prima*B1 + y_prima*B2
    v = H * 0.5 + CY + y_prima*F

    return np.array([u,v])


# Prueba 
aa = camera_model(punto)
print(aa)

[55.83150391 31.78246644]


## Camera calibration-undistort